In [1]:
import os
import shutil

if not os.path.exists("pytorch_unet.py"):
    if not os.path.exists("pytorch_unet"):
        !git clone https://github.com/akigoat/pytorch-unet.git

    # %cd pytorch-unet


import pandas as pd
import numpy as np
import random
from tqdm import tqdm, trange
import math
# files
import requests
from io import BytesIO
from zipfile import ZipFile
import os
import shutil
from pathlib import Path
# matplot
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import colors
from sklearn.metrics import mean_squared_error
# GIS
from osgeo import gdal
import rasterio
from affine import Affine
import richdem as rd
from rasterio.fill import fillnodata
# torch and dataset
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models

# functions
import sys
sys.path.append('pytorch-unet')
import helper

import dataFunctions
import netClasses

try:
    device = torch.device("mps")
except:
    print("Check device")
    print("available: ", torch.backends.mps.is_available())
    print("built: ", torch.backends.mps.is_built())

fatal: destination path 'pytorch-unet' already exists and is not an empty directory.


In [2]:
path_folder = '/Volumes/HydesT7/Grad Project/Data/U_Net/64_64_1_8_reselected/'
# path_folder = '/Users/hyde-mbp/Projects/Grad/data/64_64_1_8_rebuild/'
# path_model_folder = '/Users/hyde-mbp/Projects/Grad/model/0330/'
# path_model_folder = '/Volumes/HydesT7/Grad Project/Model/U_Net/1013/'
path_model_folder = '/Users/hyde-mbp/Projects/Grad/model/0405/'
path_ANH4 = '/Volumes/HydesT7/Grad Project/Data/AHN4_all/'
# path_pre = '/Volumes/HydesT7/Grad Project/Data/AHN4_preprocessed_1_8/'

means = np.load(path_folder+'para_means.npy')
stds = np.load(path_folder+'para_stds.npy')

# use the same transformations for train/val in this example
trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([means[0], means[1], means[2]], [
                         stds[0], stds[1], stds[2]])  # imagenet
])

list_patch = []
with open(path_ANH4+'test_patchs.txt') as fp:
    # list_patch.append(patch)
    list_patch = [line.rstrip('\n') for line in fp]

display(means, stds)

array([9.90327775, 2.48690886, 1.35894729])

array([19.5961904 ,  2.92635182,  1.85182111])

In [3]:
def mergeImg(im_list, im_dict, mask=False):
    height = int(im_dict[list(im_dict)[-1]].split(',')[0].split(':')[1])
    width = int(im_dict[list(im_dict)[-1]].split(',')[1].split(':')[1])
    im_merged = np.zeros((height, width), dtype=np.float32)
    if not mask:
        ndv = 3.4028230607370965e+38
        im_merged = im_merged-ndv
    else:
        im_merged = im_merged+0.5
    for k in im_dict:
        h0 = int(im_dict[k].split(',')[0].split(':')[0])
        h1 = int(im_dict[k].split(',')[0].split(':')[1])
        w0 = int(im_dict[k].split(',')[1].split(':')[0])
        w1 = int(im_dict[k].split(',')[1].split(':')[1])
        if not mask:
            im_merged[h0:h1, w0:w1] = np.where(
                im_list[k] > im_merged[h0:h1, w0:w1], im_list[k], im_merged[h0:h1, w0:w1])
        else:
            im_merged[h0:h1, w0:w1] = np.where(abs(
                im_list[k]-0.5) > abs(im_merged[h0:h1, w0:w1]-0.5), im_list[k], im_merged[h0:h1, w0:w1])
    return im_merged

In [4]:
model = netClasses.ResNetUNet(2)
model = model.to(device)
# /Users/hyde-mbp/Projects/Grad/model/0330/dsm_rou_slo_30_1e-4.pt
model.load_state_dict(torch.load(path_model_folder+'30_1e-4.pt'))
model.eval()


/Users/hyde-mbp/mambaforge/envs/torG2/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hyde-mbp/mambaforge/envs/torG2/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNetUNet(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

In [5]:
def dsm2dtm(dsm):
    tmp_path = 'data/tmpp/'
    cut_height = 64
    cut_width = 64

    Path(tmp_path).mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(tmp_path):
        file_path = os.path.join(tmp_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

    with rasterio.open(dsm) as src:
        profile = src.profile
        arr = src.read(1)
        arr_filled = fillnodata(arr, mask=src.read_masks(1),
                                smoothing_iterations=0)

    dsmf = tmp_path+'filledDSM.TIF'
    with rasterio.open(dsmf, 'w', **profile) as dest:
        dest.write_band(1, arr_filled)

    slope_ = tmp_path+'slope.TIF'
    _ = gdal.DEMProcessing(slope_, dsmf, 'slope', computeEdges=True)
    _ = None
    # if checkNodata(slope_):
    #     list_hasnodata.append(slope_)

    roughness_ = tmp_path+'Roughness.TIF'
    _ = gdal.DEMProcessing(roughness_, dsmf, 'Roughness', computeEdges=True)
    _ = None

    dsm_fill_slices, dsm_dict = dataFunctions.sliceRisPic(dataFunctions.readTIFF(tmp_path, 'filledDSM.TIF'), cut_height, cut_width, dt=True)
    slope_slices = dataFunctions.sliceRisPic(dataFunctions.readTIFF(tmp_path, 'slope.TIF'), cut_height, cut_width)
    roughness_slices = dataFunctions.sliceRisPic(dataFunctions.readTIFF(tmp_path, 'Roughness.TIF'), cut_height, cut_width)
    
    Imgs = dataFunctions.stackPics(dsm_fill_slices, roughness_slices, slope_slices)

    test_dataset = netClasses.TestDataset2(Imgs, transform=trans)
    test_loader = DataLoader(test_dataset, batch_size=Imgs.shape[0],
                             shuffle=False, num_workers=0)

    inputs, labels = next(iter(test_loader))
    inputs = inputs.to(device)
    labels = labels.to(device)


    pred = model(inputs)
    pred = torch.sigmoid(pred)
    pred = pred.data.cpu().numpy()
    threshold, upper, lower = 0.5, 1, 0
    pred = np.where(pred > threshold, upper, lower)

    pred_l1 = []
    for p in list(pred):
        pred_l1.append(p[1])
    pred_w = mergeImg(pred_l1, dsm_dict, mask=True)

    dtm_re = rd.LoadGDAL(dsm)

    for i in range(pred_w.shape[0]):
        for j in range(pred_w.shape[1]):
            if pred_w[i][j] > 0.5:
                dtm_re[i][j] = dtm_re.no_data
                # print(dsm_rd[i][j])


    return dtm_re, pred_w

In [6]:
def errorAnalysis(dsm, dtm, padding=False):
    tmp_path = 'data/tmpp/'
    cut_height = 64
    cut_width = 64


    Path(tmp_path).mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(tmp_path):
        file_path = os.path.join(tmp_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

    _ = shutil.copyfile(dsm, os.path.join(tmp_path, 'DSM.TIF'))
    _ = shutil.copyfile(dtm, os.path.join(tmp_path, 'DTM.TIF'))
    
    with rasterio.open(dsm) as src:
        profile = src.profile
        arr = src.read(1)
        arr_filled = fillnodata(arr, mask=src.read_masks(1),
                                smoothing_iterations=0)

    dsmf = tmp_path+'filledDSM.TIF'
    with rasterio.open(dsmf, 'w', **profile) as dest:
        dest.write_band(1, arr_filled)

    slope_ = tmp_path+'slope.TIF'
    _ = gdal.DEMProcessing(slope_, dsmf, 'slope', computeEdges=True)
    _ = None
    # if checkNodata(slope_):
    #     list_hasnodata.append(slope_)

    roughness_ = tmp_path+'Roughness.TIF'
    _ = gdal.DEMProcessing(roughness_, dsmf, 'Roughness', computeEdges=True)
    _ = None

    dsm_fill_slices, dsm_dict = dataFunctions.sliceRisPic(dataFunctions.readTIFF(tmp_path, 'filledDSM.TIF', padding=padding), cut_height, cut_width, dt=True)
    slope_slices = dataFunctions.sliceRisPic(dataFunctions.readTIFF(tmp_path, 'slope.TIF', padding=padding), cut_height, cut_width)
    roughness_slices = dataFunctions.sliceRisPic(dataFunctions.readTIFF(tmp_path, 'Roughness.TIF', padding=padding), cut_height, cut_width)

    dsm_slices = dataFunctions.sliceRisPic(dataFunctions.readTIFF(tmp_path, 'DSM.TIF', masked=True, padding=padding), cut_width, cut_height)
    dtm_slices = dataFunctions.sliceRisPic(dataFunctions.readTIFF(tmp_path, 'DTM.TIF', masked=True, padding=padding), cut_width, cut_height)
    
    Imgs = dataFunctions.stackPics(dsm_fill_slices, roughness_slices, slope_slices)

    ground_truth_slices = []
    for nn in range(len(dsm_slices)):
        ground_truth_slices.append(dataFunctions.generateGroundTruth(dsm_slices[nn], dtm_slices[nn], 0.5))
    
    Masks = np.stack(ground_truth_slices)
    Masks = Masks.astype(np.float32)
    
    gt = []
    for mask in ground_truth_slices:
        gt.append(mask[1])
    gt_w = mergeImg(gt, dsm_dict, mask=True)
    
    test_dataset = netClasses.TestDataset2(Imgs, transform=trans)
    test_loader = DataLoader(test_dataset, batch_size=Imgs.shape[0],
                             shuffle=False, num_workers=0)

    inputs, labels = next(iter(test_loader))
    inputs = inputs.to(device)
    labels = labels.to(device)


    pred = model(inputs)
    pred = torch.sigmoid(pred)
    pred = pred.data.cpu().numpy()
    threshold, upper, lower = 0.5, 1, 0
    pred = np.where(pred > threshold, upper, lower)

    pred_l1 = []
    for p in list(pred):
        pred_l1.append(p[1])
    pred_w = mergeImg(pred_l1, dsm_dict, mask=True)

    dtm_re = rd.LoadGDAL(dsm)

    for i in range(dtm_re.shape[0]):
        for j in range(dtm_re.shape[1]):
            if pred_w[i][j] > 0.5:
                dtm_re[i][j] = dtm_re.no_data
                # print(dsm_rd[i][j])

    dsm_ = dataFunctions.readTIFF(tmp_path, 'DSM.TIF', masked=True)
    dtm_ = dataFunctions.readTIFF(tmp_path, 'DTM.TIF', masked=True)

    Ndsm = dsm_.mask == True
    Ndtm = dtm_.mask == True

    mask_error = np.abs(pred_w - gt_w) > 0.1
    error_count = np.sum(mask_error)
    # error_wrong_judge = np.sum(mask_error & Ndsm)
    error_wrong_judge = np.sum(mask_error & Ndtm)
    error_ele = np.sum(mask_error & ~Ndsm & ~Ndtm)
    # bias_ele = (dsm_.data - dtm_.data)[mask_error & ~Ndsm & ~Ndtm]

    acc_count = np.sum(~mask_error)

    dic_ = {'acc': acc_count/(pred_w.shape[0]*pred_w.shape[1]), 
            'error_count': error_count, 
            'error_wrong_judge': error_wrong_judge,  
            'error_ele': error_ele}

    return pred_w, gt_w, dtm_re, dic_

In [7]:
def diffRes(path_ANH4, name, folder_out, res=30, padding=False):
    Path(folder_out).mkdir(parents=True, exist_ok=True)
    Path(os.path.join(folder_out, str(res))).mkdir(parents=True, exist_ok=True)
    tmp_path = 'data/tmppp/'
    Path(tmp_path).mkdir(parents=True, exist_ok=True)
    
    xres = res
    yres = res
    resample_alg = 'average'

    dsm = path_ANH4+'R5_'+name+'.TIF'
    dtm = path_ANH4+'M5_'+name+'.TIF'

    dsmre = tmp_path+'R'+str(res)+'_'+name+'.TIF'
    _ = gdal.Warp(dsmre, dsm, xRes=xres, yRes=yres, resampleAlg=resample_alg)
    _ = None

    dtmre = tmp_path+'M'+str(res)+'_'+name+'.TIF'
    _ = gdal.Warp(dtmre, dtm, xRes=xres, yRes=yres, resampleAlg=resample_alg)
    _ = None    
    
    pred_w, gt_w, dtm_re, dic_ = errorAnalysis(dsmre, dtmre, padding=padding)
    rdarr_dsmre = rd.LoadGDAL(dsmre)
    pred_w_ = pred_w[0:rdarr_dsmre.shape[0], 0:rdarr_dsmre.shape[1]]
    gt_w_ = gt_w[0:rdarr_dsmre.shape[0], 0:rdarr_dsmre.shape[1]]

    rd.SaveGDAL(os.path.join(tmp_path, name+'_DTM.TIF'), dtm_re)
    with rasterio.open(os.path.join(tmp_path, name+'_DTM.TIF')) as src:
        profile = src.profile
        arr = src.read(1)
        arr_filled = fillnodata(arr, mask=src.read_masks(1), smoothing_iterations=0)
    dtmref = os.path.join(tmp_path, name+'_DTM_filled.TIF')  
    with rasterio.open(dtmref, 'w', **profile) as dest:
        dest.write_band(1, arr_filled)

    with rasterio.open(dsmre) as src_:
        profile_ = src_.profile
        arr_ = src_.read(1)
        arr_filled_ = fillnodata(arr_, mask=src_.read_masks(1), smoothing_iterations=0)
    dsmref = os.path.join(tmp_path, name+'_DSM_filled.TIF')  
    with rasterio.open(dsmref, 'w', **profile_) as dest_:
        dest_.write_band(1, arr_filled_)
    
    with rasterio.open(dtmre) as src_:
        profile_ = src_.profile
        arr_ = src_.read(1)
        arr_filled_ = fillnodata(arr_, mask=src_.read_masks(1), smoothing_iterations=0)
    dtmref = os.path.join(tmp_path, name+'_DTM_filled.TIF')  
    with rasterio.open(dtmref, 'w', **profile_) as dest_:
        dest_.write_band(1, arr_filled_)    
    
    

    return dtm_re
    

In [25]:
_ = diffRes(path_ANH4=path_ANH4, name='10EN2', folder_out='result_otherRes', res=90, padding=True)
_.shape

(69, 56)

In [9]:
def getDtmNPlots(cop, fabdem, name, folder_out):
    Path(folder_out).mkdir(parents=True, exist_ok=True)

    dtm_re, pred_w = dsm2dtm(cop)

    rd.SaveGDAL(os.path.join(folder_out, name+'_DTM.TIF'), dtm_re)
    with rasterio.open(os.path.join(folder_out, name+'_DTM.TIF')) as src:
        profile = src.profile
        arr = src.read(1)
        arr_filled = fillnodata(arr, mask=src.read_masks(1), smoothing_iterations=0)
    dtmf = os.path.join(folder_out, name+'_DTM_filled.TIF')  
    with rasterio.open(dtmf, 'w', **profile) as dest:
        dest.write_band(1, arr_filled)
    

    fig = plt.figure(figsize=(12, 12))
    rda_cop = rd.LoadGDAL(cop)
    disparr_cop = np.array(rda_cop, copy=True)
    disparr_cop[disparr_cop < rda_cop.no_data/10] = np.nan
    vmin, vmax = np.nanpercentile(disparr_cop, [2, 98])
    rda_fabdem = rd.LoadGDAL(fabdem)
    disparr_fabdem = np.array(rda_fabdem, copy=True)
    disparr_fabdem[disparr_fabdem < rda_fabdem.no_data/10] = np.nan
    rda_dtmf = rd.LoadGDAL(dtmf)
    disparr_dtmf = np.array(rda_dtmf, copy=True)
    disparr_dtmf[disparr_dtmf < rda_dtmf.no_data/10] = np.nan


    ax1 = fig.add_subplot(221)
    im1 = ax1.imshow(disparr_cop, vmin=vmin, vmax=vmax, cmap='terrain')
    ax1.set_xlim()
    ax1.set_ylim()
    ax1.set_title('COP30')
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im1, cax=cax, orientation='vertical')

    ax2 = fig.add_subplot(222)
    im2 = ax2.imshow(disparr_fabdem, vmin=vmin, vmax=vmax, cmap='terrain')
    ax2.set_xlim()
    ax2.set_ylim()
    ax2.set_title('FABDEM')
    divider = make_axes_locatable(ax2)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im2, cax=cax, orientation='vertical')

    ax3 = fig.add_subplot(223)
    im3 = ax3.imshow(disparr_dtmf, vmin=vmin, vmax=vmax, cmap='terrain')
    ax3.set_xlim()
    ax3.set_ylim()
    ax3.set_title('DTM generated by the model')
    divider = make_axes_locatable(ax3)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im3, cax=cax, orientation='vertical')

    color = [[255, 197, 77], [83, 191, 157]]
    color = np.array(color)/255
    cmap = colors.ListedColormap(color)
    norm = colors.BoundaryNorm([0, 0.5, 1], 2, clip=True)
    ax4 = fig.add_subplot(224)
    im4 = ax4.imshow(pred_w, cmap=cmap, norm=norm)
    ax4.set_title('Prediction')
    divider = make_axes_locatable(ax4)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im4, cax=cax, orientation='vertical', ticks=np.linspace(0, 1, 2))

    plt.tight_layout()
    plt.savefig(os.path.join(folder_out, name+'_f4.png'))

    extent1 = ax1.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(os.path.join(folder_out, name+'_COP30.png'), bbox_inches=extent1.expanded(1.3, 1.1))
    extent2 = ax2.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(os.path.join(folder_out, name+'_FABDEM.png'), bbox_inches=extent2.expanded(1.3, 1.1))
    extent3 = ax3.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(os.path.join(folder_out, name+'_DTMf.png'), bbox_inches=extent3.expanded(1.3, 1.1))
    extent4 = ax4.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(os.path.join(folder_out, name+'_Pred.png'), bbox_inches=extent4.expanded(1.3, 1.1))

    plt.close()
    # rd.SaveGDAL(os.path.join(folder_out, name+'_DTM.TIF'), dtm_re)

    # with rasterio.open(os.path.join(folder_out, name+'_DTM.TIF')) as src:
    #     profile = src.profile
    #     arr = src.read(1)
    #     arr_filled = fillnodata(arr, mask=src.read_masks(1), smoothing_iterations=0)

    # # dtm30ref = tmp_path+'M30_'+item+'_re_filled.TIF'  
    # with rasterio.open(os.path.join(folder_out, name+'_DTM_filled.TIF'), 'w', **profile) as dest:
    #     dest.write_band(1, arr_filled)


    return f'Plots are saved as {folder_out}/{name}_f4.png'

In [59]:
def rda2arr(rda):
    no_data = rda.no_data
    arr = np.array(rda, copy=True)
    if no_data>0:
        arr[arr>no_data/10] = np.nan
    else:
        arr[arr<no_data/10] = np.nan
    return arr

In [93]:
def rmseNMore(path_cop, path_fabdem, path_ANH4, name, folder_out):
    Path(folder_out).mkdir(parents=True, exist_ok=True)
    Path(os.path.join(folder_out, 'cop')).mkdir(parents=True, exist_ok=True)
    Path(os.path.join(folder_out, 'AHN4')).mkdir(parents=True, exist_ok=True)
    Path(os.path.join(folder_out, 'fabdem')).mkdir(parents=True, exist_ok=True)
    Path(os.path.join(folder_out, 'pred')).mkdir(parents=True, exist_ok=True)

    tmp_path = 'data/tmp/'
    xres = 30
    yres = 30
    resample_alg = 'average'
    # cut_height = 64
    # cut_width = 64

    Path(tmp_path).mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(tmp_path):
        file_path = os.path.join(tmp_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

    cop = os.path.join(path_cop, name+'.tiff')
    fabdem = os.path.join(path_fabdem, name+'.tiff')

    dsm5 = path_ANH4+'R5_'+name+'.TIF'
    dtm5 = path_ANH4+'M5_'+name+'.TIF'

    dsm30 = tmp_path+'R30_'+name+'.TIF'
    _ = gdal.Warp(dsm30, dsm5, xRes=xres, yRes=yres, resampleAlg=resample_alg)
    _ = None

    with rasterio.open(dsm30) as src:
        profile = src.profile
        arr = src.read(1)
        arr_filled = fillnodata(arr, mask=src.read_masks(1),
                                smoothing_iterations=0)

    dsm30f = tmp_path+'R30_'+name+'_filled.TIF'
    with rasterio.open(dsm30f, 'w', **profile) as dest:
        dest.write_band(1, arr_filled)


    dtm30 = tmp_path+'M30_'+name+'.TIF'
    _ = gdal.Warp(dtm30, dtm5, xRes=xres, yRes=yres, resampleAlg=resample_alg)
    _ = None

    with rasterio.open(dtm30) as src:
        profile = src.profile
        arr = src.read(1)
        arr_filled = fillnodata(arr, mask=src.read_masks(1),
                                smoothing_iterations=0)

    dtm30f = tmp_path+'M30_'+name+'_filled.TIF'
    with rasterio.open(dtm30f, 'w', **profile) as dest_:
        dest_.write_band(1, arr_filled)
    

    dtm_A, pred_A = dsm2dtm(dsm30f)
    dtm_cop, pred_cop = dsm2dtm(cop)

    rd.SaveGDAL(os.path.join(folder_out, name+'_DTM.TIF'), dtm_A)
    with rasterio.open(os.path.join(folder_out, name+'_DTM.TIF')) as src:
        profile = src.profile
        arr = src.read(1)
        arr_filled = fillnodata(arr, mask=src.read_masks(1), smoothing_iterations=0)
    dtmf = os.path.join(folder_out, name+'_DTM_filled.TIF')  
    with rasterio.open(dtmf, 'w', **profile) as dest:
        dest.write_band(1, arr_filled)

    rd.SaveGDAL(os.path.join(folder_out, name+'_DTM_COP.TIF'), dtm_cop)
    with rasterio.open(os.path.join(folder_out, name+'_DTM_COP.TIF')) as src_:
        profile_ = src_.profile
        arr_ = src_.read(1)
        arr_filled_ = fillnodata(arr_, mask=src_.read_masks(1), smoothing_iterations=0)
    dtmfc = os.path.join(folder_out, name+'_DTM_COP_filled.TIF')  
    with rasterio.open(dtmfc, 'w', **profile_) as dest_:
        dest_.write_band(1, arr_filled_)

    fig = plt.figure(figsize=(12, 24))

    # rda_cop = rd.LoadGDAL(cop)
    # disparr_cop = np.array(rda_cop, copy=True)
    # disparr_cop[disparr_cop < rda_cop.no_data/10] = np.nan
    narr_cop = rda2arr(rd.LoadGDAL(cop))
    # vmin, vmax = np.nanpercentile(narr_cop, [2, 98])
    narr_dsmf = rda2arr(rd.LoadGDAL(dsm30f))
    vmin, vmax = np.nanpercentile(np.concatenate((narr_cop.ravel(), narr_dsmf.ravel())), [2, 98])
    narr_dtmf = rda2arr(rd.LoadGDAL(dtmf))
    narr_dtmfc = rda2arr(rd.LoadGDAL(dtmfc))
    narr_dtmAf = rda2arr(rd.LoadGDAL(dtm30f))
    narr_fabdem = rda2arr(rd.LoadGDAL(fabdem))

    ax1 = fig.add_subplot(421)
    im1 = ax1.imshow(narr_dsmf, vmin=vmin, vmax=vmax, cmap='terrain')
    ax1.set_xlim()
    ax1.set_ylim()
    ax1.set_title('AHN4 DSM (resampled to 30m)')
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im1, cax=cax, orientation='vertical')

    ax2 = fig.add_subplot(422)
    im2 = ax2.imshow(narr_dtmf, vmin=vmin, vmax=vmax, cmap='terrain')
    ax2.set_xlim()
    ax2.set_ylim()
    ax2.set_title('DTM generated by the model')
    divider = make_axes_locatable(ax2)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im2, cax=cax, orientation='vertical')

    ax3 = fig.add_subplot(423)
    im3 = ax3.imshow(narr_cop, vmin=vmin, vmax=vmax, cmap='terrain')
    ax3.set_xlim()
    ax3.set_ylim()
    ax3.set_title('COP30')
    divider = make_axes_locatable(ax3)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im3, cax=cax, orientation='vertical')

    ax4 = fig.add_subplot(424)
    im4 = ax4.imshow(narr_dtmfc, vmin=vmin, vmax=vmax, cmap='terrain')
    ax4.set_xlim()
    ax4.set_ylim()
    ax4.set_title('DTM generated by the model')
    divider = make_axes_locatable(ax4)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im4, cax=cax, orientation='vertical')

    ax5 = fig.add_subplot(425)
    im5 = ax5.imshow(narr_dtmAf, vmin=vmin, vmax=vmax, cmap='terrain')
    ax5.set_xlim()
    ax5.set_ylim()
    ax5.set_title('AHN4 DTM (resampled to 30m)')
    divider = make_axes_locatable(ax5)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im5, cax=cax, orientation='vertical')

    ax6 = fig.add_subplot(426)
    im6 = ax6.imshow(narr_fabdem, vmin=vmin, vmax=vmax, cmap='terrain')
    ax6.set_xlim()
    ax6.set_ylim()
    ax6.set_title('FABDEM')
    divider = make_axes_locatable(ax6)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im6, cax=cax, orientation='vertical')

    color = [[255, 197, 77], [83, 191, 157]]
    color = np.array(color)/255
    cmap = colors.ListedColormap(color)
    norm = colors.BoundaryNorm([0, 0.5, 1], 2, clip=True)

    ax7 = fig.add_subplot(427)
    im7 = ax7.imshow(pred_A, cmap=cmap, norm=norm)
    ax7.set_title('Prediction of AHN4 DSM')
    divider = make_axes_locatable(ax7)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im7, cax=cax, orientation='vertical', ticks=np.linspace(0, 1, 2))

    ax8 = fig.add_subplot(428)
    im8 = ax8.imshow(pred_cop, cmap=cmap, norm=norm)
    ax8.set_title('Prediction of COP30')
    divider = make_axes_locatable(ax8)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im8, cax=cax, orientation='vertical', ticks=np.linspace(0, 1, 2))

    plt.tight_layout()
    plt.savefig(os.path.join(folder_out, name+'_f8.png'))

    extent1 = ax1.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(os.path.join(os.path.join(folder_out, 'AHN4'), name+'_DSM.png'), bbox_inches=extent1.expanded(1.3, 1.1))
    extent2 = ax2.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(os.path.join(os.path.join(folder_out, 'AHN4'), name+'_DTMg.png'), bbox_inches=extent2.expanded(1.3, 1.1))
    extent3 = ax3.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(os.path.join(os.path.join(folder_out, 'cop'), name+'_COP30.png'), bbox_inches=extent3.expanded(1.3, 1.1))
    extent4 = ax4.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(os.path.join(os.path.join(folder_out, 'cop'), name+'_copDTM.png'), bbox_inches=extent4.expanded(1.3, 1.1))   
    extent5 = ax5.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(os.path.join(os.path.join(folder_out, 'AHN4'), name+'_DTMo.png'), bbox_inches=extent5.expanded(1.3, 1.1))
    extent6 = ax6.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(os.path.join(os.path.join(folder_out, 'fabdem'), name+'_fab.png'), bbox_inches=extent6.expanded(1.3, 1.1))
    extent7 = ax7.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(os.path.join(os.path.join(folder_out, 'pred'), name+'_AHN4.png'), bbox_inches=extent7.expanded(1.3, 1.1))
    extent8 = ax8.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(os.path.join(os.path.join(folder_out, 'pred'), name+'_COP30.png'), bbox_inches=extent8.expanded(1.3, 1.1))

    plt.close()

    modelRSME = np.sqrt(mean_squared_error(narr_dtmAf, narr_dtmf))
    fabdemRSME = np.sqrt(mean_squared_error(narr_fabdem, narr_dtmf[0:narr_fabdem.shape[0], 0:narr_fabdem.shape[1]]))
    modelCRMSE = np.sqrt(mean_squared_error(narr_dtmfc, narr_dtmf[0:narr_dtmfc.shape[0], 0:narr_dtmfc.shape[1]]))

    dt = {'modelRSME': modelRSME, 'fabdemRSME': fabdemRSME, 'modelCRMSE': modelCRMSE}

    return dt

In [95]:
dt_RMSE = {}
pbar = tqdm(total=len(list_patch))
for item in list_patch:
    dt_RMSE[item]=rmseNMore(path_cop='/Users/hyde-mbp/Projects/Grad/data/COP30/', 
          path_ANH4=path_ANH4, 
          path_fabdem='/Users/hyde-mbp/Projects/Grad/data/FABDEM/', 
          name=item, 
          folder_out='results_RMSE')
    pbar.update(1)

pbar.close()

100%|██████████| 50/50 [02:28<00:00,  2.96s/it]


TypeError: Object of type float32 is not JSON serializable

In [101]:
import json

class NumpyFloatValuesEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.float32):
            return float(obj)
        return JSONEncoder.default(self, obj)

with open("results_RMSE/RMSEs.json", "w") as outfile:
    json.dump(dt_RMSE, outfile, cls=NumpyFloatValuesEncoder, indent=2)

In [103]:
min_values = {}
max_values = {}
sum_values = {key: 0 for key in dt_RMSE[next(iter(dt_RMSE))].keys()}

# Iterate through the dictionary to find min, max, and sum values for each RMSE
for key, sub_dict in dt_RMSE.items():
    for sub_key, value in sub_dict.items():
        if sub_key not in min_values or value < min_values[sub_key][1]:
            min_values[sub_key] = (key, value)
        if sub_key not in max_values or value > max_values[sub_key][1]:
            max_values[sub_key] = (key, value)
        sum_values[sub_key] += value

# Calculate the mean values
mean_values = {key: value / len(dt_RMSE) for key, value in sum_values.items()}

print("Mean values:", mean_values)
print("Min values:", min_values)
print("Max values:", max_values)

Mean values: {'modelRSME': 0.6008902508020401, 'fabdemRSME': 0.9341578716039658, 'modelCRMSE': 1.1768759590387345}
Min values: {'modelRSME': ('10EN2', 0.12665087), 'fabdemRSME': ('65GZ2', 0.37494177), 'modelCRMSE': ('10FZ2', 0.39659908)}
Max values: {'modelRSME': ('25CN1', 1.7051184), 'fabdemRSME': ('25CN1', 2.9643419), 'modelCRMSE': ('49GN1', 3.7635415)}


In [10]:
getDtmNPlots(cop='/Users/hyde-mbp/Projects/Grad/data/COP30/Austria_re2.tiff',
            fabdem='/Users/hyde-mbp/Projects/Grad/data/FABDEM/Austria_re2.tiff', 
            name='Austria', 
            folder_out='result_other_areas')

'Plots are saved as result_other_areas/Austria_f4.png'